In [1]:
%%writefile pergola.yaml
version: v1

components:
- name: todo-app
  docker:
    file: Dockerfile
  ports:
  - 5050
  ingresses:
  - host: app
  env:
  - name: SQLALCHEMY_HOST
    component-ref: todo-db
  - name: SQLALCHEMY_PORT
    value: "5432"
  - name: SQLALCHEMY_DB
    value: flask-demo
  - name: SQLALCHEMY_USER
    config-ref: DB_USER
  - name: SQLALCHEMY_PASSWORD
    config-ref: DB_PASS

- name: todo-db
  docker:
    image: postgres:14.2
  env:
  - name: POSTGRES_DB
    value: flask-demo
  - name: POSTGRES_USER
    config-ref: DB_USER
  - name: POSTGRES_PASSWORD
    config-ref: DB_PASS
  storage:
  - name: pgdata
    path: /var/lib/postgresql/data
    size: 1Gi
  ports:
  - 5432

Writing pergola.yaml


In [2]:
%%writefile Dockerfile
FROM python:3.8-slim

RUN apt-get update -y && \
    apt-get install -y gcc libpq-dev && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

COPY requirements.txt requirements.txt
RUN pip install -r requirements.txt

WORKDIR /todo-list

COPY test.py test.py

ENV FLASK_APP=app
CMD flask run --host=0.0.0.0 --port 5050

Writing Dockerfile


In [19]:
%%writefile docker-compose.yml
version: '3'
services:
  flask-todo-web:
    build: .
    environment:
      SQLALCHEMY_HOST: flask-todo-db
      SQLALCHEMY_PORT: "5432"
      SQLALCHEMY_DB: flask-demo
      SQLALCHEMY_USER: flask-demo
      SQLALCHEMY_PASSWORD: flask-demo
    ports:
    - "5050:5050"
    depends_on:
    - flask-todo-db

  flask-todo-db:
    image: postgres:14.2
    environment:
      POSTGRES_USER: flask-demo
      POSTGRES_PASSWORD: flask-demo
      POSTGRES_DB: flask-demo
    healthcheck:
      test: ["CMD-SHELL", "pg_isready -h localhost -U flask-demo"]
      interval: 2s
      timeout: 5s
      retries: 5
    ports:
    - "5432:5432"

Overwriting docker-compose.yml


In [12]:
!ls

Dockerfile                     pergola.yaml
README                         pergola_getting_familiar.ipynb
docker-compose.yml             requirements.txt
olip78.git                     test.py


In [60]:
%%sh
git init
git pull
git add --all
git commit -m "add config files test2"
git branch -M main
git remote -v
git push -u origin main

Reinitialized existing Git repository in /Users/andreichekunov/brandcompete/sagemaker/pergola_deployment/test/.git/


There is no tracking information for the current branch.
Please specify which branch you want to merge with.
See git-pull(1) for details.

    git pull <remote> <branch>

If you wish to set tracking information for this branch you can do so with:

    git branch --set-upstream-to=origin/<branch> main



On branch main
nothing to commit, working tree clean
origin	https://github.com/olip78/pergola_test.git (fetch)
origin	https://github.com/olip78/pergola_test.git (push)


To https://github.com/olip78/pergola_test.git
 ! [rejected]        main -> main (non-fast-forward)
error: failed to push some refs to 'https://github.com/olip78/pergola_test.git'
hint: Updates were rejected because the tip of your current branch is behind
hint: its remote counterpart. Integrate the remote changes (e.g.
hint: 'git pull ...') before pushing again.
hint: See the 'Note about fast-forwards' in 'git push --help' for details.


CalledProcessError: Command 'b'git init\ngit pull\ngit add --all\ngit commit -m "add config files test2"\ngit branch -M main\ngit remote -v\ngit push -u origin main\n'' returned non-zero exit status 1.

In [57]:
!git push --help

GIT-PUSH(1)                        Git Manual                        GIT-PUSH(1)



NAME
       git-push - Update remote refs along with associated objects

SYNOPSIS
       git push [--all | --mirror | --tags] [--follow-tags] [--atomic] [-n | --dry-run] [--receive-pack=<git-receive-pack>]
                  [--repo=<repository>] [-f | --force] [-d | --delete] [--prune] [-v | --verbose]
                  [-u | --set-upstream] [-o <string> | --push-option=<string>]
                  [--[no-]signed|--signed=(true|false|if-asked)]
                  [--force-with-lease[=<refname>[:<expect>]] [--force-if-includes]]
                  [--no-verify] [<repository> [<refspec>...]]


DESCRIPTION
       Updates remote refs using local refs, while sending objects necessary to
       complete the given refs.

       You can make interesting things happen to a repository every time you
       push into it, by setting up hooks there. See documentation for git-
       receive-pack(1).

       When the co